In [18]:
import json, requests, csv
import pandas as pd
import csv, re, time
from datetime import datetime
from textblob import TextBlob
import numpy as np
from bs4 import BeautifulSoup
from dateutil.parser import parse
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
import math

In [19]:
# creation du fichier company

url_company = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/Company-2024-03-03.csv'
data_company = pd.read_csv(url_company)


# creation du fichier lookup business

url_Googlebusinesslookup = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GooglebusinessLookup-2024-03-04.csv'
Googlebusinesslookup = pd.read_csv(url_Googlebusinesslookup)

# creation du fichier lookup business

url_Googlebusinessmatched = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GooglebusinessMatched-2024-03-04.csv'
Googlebusinessmatched = pd.read_csv(url_Googlebusinessmatched)

# creation du fichier reviews

url_GooglebusinessReviews = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GooglebusinessReviews-2024-03-04.csv'
GooglebusinessReviews = pd.read_csv(url_GooglebusinessReviews)

In [20]:
company_list = []
SIRET_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in data_company.iterrows():
    company = occurrence['id']  # Assurez-vous que 'id' est le nom correct de la colonne
    SIRET = occurrence['siret']  # Assurez-vous que 'siret' est le nom correct de la colonne
    company_list.append(company)
    SIRET_list.append(SIRET)

Id_SIRET = dict(zip(company_list, SIRET_list))

In [21]:
Googlebusinesslookup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     141 non-null    int64  
 1   company                141 non-null    int64  
 2   google_results         135 non-null    object 
 3   local_results          0 non-null      float64
 4   scraped                135 non-null    float64
 5   scraped_date           135 non-null    object 
 6   scrap_log              0 non-null      float64
 7   scrap_cost             141 non-null    int64  
 8   extracted              134 non-null    float64
 9   name_content           38 non-null     object 
 10  address_content        36 non-null     object 
 11  rating_content         32 non-null     float64
 12  reviews_content        33 non-null     float64
 13  other_reviews_content  5 non-null      object 
 14  website_content        23 non-null     object 
 15  review

In [22]:
Googlebusinessmatched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     140 non-null    int64  
 1   company                140 non-null    int64  
 2   google_results         140 non-null    object 
 3   local_results          0 non-null      float64
 4   scraped                140 non-null    int64  
 5   scraped_date           140 non-null    object 
 6   scrap_log              0 non-null      float64
 7   scrap_cost             140 non-null    int64  
 8   name_content           140 non-null    object 
 9   address_content        140 non-null    object 
 10  rating_content         133 non-null    float64
 11  reviews_content        133 non-null    float64
 12  other_reviews_content  44 non-null     object 
 13  website_content        121 non-null    object 
 14  reviews_url_content    140 non-null    object 
 15  lat_co

In [23]:
GooglebusinessReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    140 non-null    int64  
 1   matched               140 non-null    int64  
 2   html_content          140 non-null    object 
 3   scraped               140 non-null    int64  
 4   scraped_date          140 non-null    object 
 5   scrap_cost            140 non-null    int64  
 6   scrap_log             0 non-null      float64
 7   extracted             140 non-null    int64  
 8   review_results        117 non-null    object 
 9   review_results_count  140 non-null    int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 11.1+ KB


In [41]:
def formater_chaine(chaine):
    lignes = []
    paragraphe = ""
    index_debut = chaine.find("'name'")
    while index_debut != -1:
        index_fin = chaine.find(", 'text'", index_debut)
        if index_fin != -1:
            chaine = chaine[:index_debut] + chaine[index_fin:]
        else:
            break
        # Rechercher la prochaine occurrence de "'name'"
        index_debut = chaine.find("'name'")

    chaine = chaine.replace("},{", "\n")
    chaine = chaine.replace("},", "\n")
    chaine = chaine.replace("[{", "")
    chaine = chaine.replace("}]", "")
    chaine = chaine.replace('date','\n"date:"')
    chaine = chaine.replace('text','\n"avis:"')
    chaine = chaine.replace('rating','\n"évaluation:"')
    chaine =chaine.replace(", '","")
    for carac in chaine:
        if carac == "{":
            lignes.append(paragraphe)
            paragraphe = ""
        else:
            paragraphe += carac
    if paragraphe:
        lignes.append(paragraphe)
    return '\n'.join(lignes)

def unicode_to_char(text):
    if text is None:
        return ""
    try:
        return text.encode('latin1', errors='replace').decode('unicode_escape')
    except UnicodeDecodeError:
        # Vous pouvez renvoyer text sans modification,
        # ou peut-être effectuer une autre forme de transformation/échappement
        return text
    
# Fonction pour nettoyer le texte
def clean_text(text):
    # Remplacer les caractères de substitution par un espace ou par un autre caractère
    # Vous pouvez choisir de supprimer le caractère ou de le remplacer par quelque chose d'autre
    return text.encode('utf-8', 'ignore').decode('utf-8')


In [31]:
company_matched_list = []
rating_content_list = []
name_content_list = []
reviews_content_list = []
id_matched_list = []
website_content_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in Googlebusinessmatched.iterrows():
    company_matched = occurrence['company']
    id_matched = occurrence['id']
    rating_content = occurrence['rating_content']
    name_content = occurrence['name_content']
    reviews_content = occurrence['reviews_content']
    website_content = occurrence['website_content']
    company_matched_list.append(company_matched)
    id_matched_list.append(id_matched)
    rating_content_list.append(rating_content)
    name_content_list.append(name_content)
    reviews_content_list.append(reviews_content)
    website_content_list.append(website_content)


In [32]:
review_results_list = []
company_google_list = []
id_google_list = []
SIRET_google_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in GooglebusinessReviews.iterrows():
        id_google = occurrence['matched']
        index_id = id_matched_list.index(id_google)
        company_google = company_matched_list[index_id]
        company_google_list.append(company_google)
        id_google_list.append(id_google)
        SIRET_google = SIRET_list[company_list.index(company_google)]
        SIRET_google_list.append(SIRET_google)
        review_results = str(occurrence['review_results'])
        review_results = formater_chaine(review_results)
        review_results = unicode_to_char(review_results)
        review_results_list.append(review_results)

In [35]:
# Créer un dictionnaire vide pour les données
donnees = {}

# Remplir le dictionnaire avec les listes de données en utilisant le SIRET comme clé
for SIRET_google, company_google, rating_content, reviews_content, review_results in zip(SIRET_google_list,
                            company_google_list,
                            rating_content_list,
                            reviews_content_list,
                            review_results_list):
    donnees[SIRET_google] = {"SIRET": SIRET_google,
                            "company": company_google,
                            "star": rating_content,
                            "avis": reviews_content,
                            "resultats": review_results}
donnees = list(donnees.values())

In [43]:
# Conversion de la liste de dictionnaires en DataFrame
google_business = pd.DataFrame(donnees)

# Appliquer la fonction clean aux données
google_business['resultats'] = google_business['resultats'].apply(clean_text)

In [44]:
# Obtenir la date actuelle
date_actuelle = datetime.now().strftime('%Y-%m-%d')  # Formate la date comme "AAAA-MM-JJ"

# Créer le nom du fichier avec la date
nom_fichier = f"google_business_{date_actuelle}.csv"

# Exporter le DataFrame en CSV
google_business.to_csv(nom_fichier, index=False)